Поиск по артисту

In [1]:
import requests, json, urllib.parse

In [2]:
base_url = 'https://itunes.apple.com/search'

artist = 'Billy Joel'

params = {
    'term': artist,
    'limit': 200,
    'media': 'music',
    'entity': 'song',
    'attribute': 'artistTerm'
}

response = requests.get(f"{base_url}?{urllib.parse.urlencode(params)}")

data = json.loads(response.text)
data = data['results']

In [3]:
print(list(data[0].keys()))

['wrapperType', 'kind', 'artistId', 'collectionId', 'trackId', 'artistName', 'collectionName', 'trackName', 'collectionCensoredName', 'trackCensoredName', 'artistViewUrl', 'collectionViewUrl', 'trackViewUrl', 'previewUrl', 'artworkUrl30', 'artworkUrl60', 'artworkUrl100', 'collectionPrice', 'trackPrice', 'releaseDate', 'collectionExplicitness', 'trackExplicitness', 'discCount', 'discNumber', 'trackCount', 'trackNumber', 'trackTimeMillis', 'country', 'currency', 'primaryGenreName', 'isStreamable']


In [4]:
import dateutil.parser
data = map(lambda x: {**x, 'releaseDate': dateutil.parser.isoparse(x['releaseDate'])}, data)
data = list(data)

Первичная очистка данных

In [5]:
data = map(lambda x: {
    'artistId': x['artistId'],
    'collectionId': x['collectionId'],
    'trackId': x['trackId'],
    'artistName': x['artistName'],
    'collectionName': x['collectionCensoredName'],
    'trackName': x['trackCensoredName'],
    'collectionPrice': x['collectionPrice'],
    'trackPrice': x['trackPrice'],
    'releaseDate': x['releaseDate'],
    'collectionExplicitness': x['collectionExplicitness'],
    'trackExplicitness': x['trackExplicitness'],
    'discCount': x['discCount'],
    'discNumber': x['discNumber'],
    'trackCount': x['trackCount'],
    'trackNumber': x['trackNumber'],
    'trackTimeMillis': x['trackTimeMillis'],
    'country': x['country'],
    'currency': x['currency'],
    'primaryGenreName': x['primaryGenreName'],
    'isStreamable': x['isStreamable']
}, data)
data = list(data)
print(list(data[0].keys()))

['artistId', 'collectionId', 'trackId', 'artistName', 'collectionName', 'trackName', 'collectionPrice', 'trackPrice', 'releaseDate', 'collectionExplicitness', 'trackExplicitness', 'discCount', 'discNumber', 'trackCount', 'trackNumber', 'trackTimeMillis', 'country', 'currency', 'primaryGenreName', 'isStreamable']


Перевод милисекунд в {часы}:{минуты}:{секунды}

In [6]:
def convert_millis(millis: int) -> str:
    total_seconds = millis // 1000
    seconds = total_seconds % 60
    minutes = (total_seconds // 60) % 60
    hours = (total_seconds // (60 * 60))
    return f"{hours}H{minutes}M{seconds}S"

Статистика по исполнителю из полученной выборки данных

In [7]:
Artist = lambda x: x['artistName']
print(set(map(Artist, data)))

{'Billy Joel'}


In [8]:
data = filter(lambda x: artist.lower() in x['artistName'].lower(), data)
data = list(data)

In [9]:
Count = lambda x: x['trackName']
Count_Of_Songs = len(list(map(Count, data)))
print('Count of songs:', Count_Of_Songs)

Count of songs: 200


In [10]:
Genre = lambda x: x['primaryGenreName']
Popular_Genre = (list((map(Genre, data))))
if Popular_Genre.count('Pop') > Popular_Genre.count('Rock'):
    print('Most popular genre:','Pop -', Popular_Genre.count('Pop'))
else:
    print('Most popular genre:','Rock -', Popular_Genre.count('Rock'))

Most popular genre: Pop - 120


In [14]:
Album = lambda x: x['collectionId']
Album_count = len(list(set(map(Album, data))))
print('Album count:', Album_count)

Album count: 20


In [15]:
time_extractor = lambda x: x['trackTimeMillis']

Duration_of_songs = sum(list(map(time_extractor, data)))
print('Duration of songs:', convert_millis(Duration_of_songs))

Average_duration = Duration_of_songs / Count_Of_Songs
print('Average duration of songs:', convert_millis(Average_duration))

Median_time = list(map(time_extractor, data))
import statistics
print('Mediana of songs:', convert_millis(statistics.median(Median_time)))

longest_song = max(data, key=time_extractor)
print('Longest song:', longest_song['trackName'], convert_millis(longest_song['trackTimeMillis']))

shortest_song = min(data, key=time_extractor)
print('Shortest song:', shortest_song['trackName'], convert_millis(shortest_song['trackTimeMillis']))


Duration of songs: 14H41M16S
Average duration of songs: 0.0H4.0M24.0S
Mediana of songs: 0.0H4.0M7.0S
Longest song: Scenes from an Italian Restaurant 0H7M35S
Shortest song: Invention In C minor 0H1M4S


In [16]:
Song_cost = lambda x: x['trackPrice']

All_price = sum(list(map(Song_cost, data)))
print('Price of all songs:', All_price, '$')

Average_price = All_price / Count_Of_Songs
print('Average price of songs:', Average_price, '$')

Median_cost = list(map(Song_cost, data))
import statistics
print('Median of songs cost:', statistics.median(Median_cost), '$')

Expensive_song = max(data, key=Song_cost)
print('Expensive song:', Expensive_song['trackName'], Expensive_song['trackPrice'],'$')

Cheap_song = min(data, key=Song_cost)
if Cheap_song['trackPrice'] == -1 : 
    print('Cheap song:', Cheap_song['trackName'],'Free')
else:
    print('Cheap song:', Cheap_song['trackName'], Cheap_song['trackPrice'], '$')

Price of all songs: 248.83999999999952 $
Average price of songs: 1.2441999999999975 $
Median of songs cost: 1.29 $
Expensive song: Piano Man 1.29 $
Cheap song: Miami 2017 (I've Seen the Lights Go Out On Broadway) [Live] Free


Статистика по альбомам исполнителя из полученной выборки данных

In [24]:
from itertools import groupby

collection_id_extractor = lambda x: x['collectionId']

sorted_by_album = sorted(data, key=collection_id_extractor)
album_groups = {}
album_names = {}

for album_id, songs in groupby(sorted_by_album, key=collection_id_extractor):
    songs = list(songs)
    album_groups[album_id] = songs 
    album_names[album_id] = songs[0]['collectionName']
del sorted_by_album

print(album_names)

{157506242: 'The Nylon Curtain', 158617297: 'Piano Man', 158617952: 'The Stranger', 158618393: 'Glass Houses', 158618758: 'Storm Front', 158619540: 'River of Dreams', 158625840: '12 Gardens Live', 158815463: 'The Essential Billy Joel', 190412464: 'Songs In the Attic (Live)', 192816208: 'The Bridge', 217360639: 'Greatest Hits, Vol. 3', 259213439: 'Turnstiles', 259573387: '52nd Street', 259814641: 'An Innocent Man', 275318699: "Now That's What I Call the 80's", 383633357: 'My Lives', 401938490: 'The Hits', 585701590: '12-12-12 The Concert for Sandy Relief', 794025628: 'The Complete Albums Collection', 1461891971: 'Greatest Hits, Volume I & Volume II'}


In [34]:
Album_name = album_names[158617297]
print('Album name:', Album_name)

Album name: Piano Man


In [33]:
Album_value = list(album_groups[158617297])

Count = lambda x: x['trackId']
Count_Of_Album_Songs = len(list(set(map(Count, Album_value))))
print('Count of songs:', Count_Of_Album_Songs)

Count of songs: 9


In [32]:
Genre = lambda x: x['primaryGenreName']
Popular_Genre = (list((map(Genre, Album_value))))
if Popular_Genre.count('Pop') > Popular_Genre.count('Rock'):
    print('Most popular genre:','Pop -', Popular_Genre.count('Pop'))
else:
    print('Most popular genre:','Rock -', Popular_Genre.count('Rock'))

Most popular genre: Rock - 9


In [31]:
release_date_extractor = lambda x: x['releaseDate']

Release_date = min(Album_value, key=release_date_extractor)
print('Release date:', Release_date['trackName'], Release_date['releaseDate'])

Release date: You're My Home 1973-11-02 08:00:00+00:00


In [30]:
Album_Song_cost = lambda x: x['trackPrice']

All_album_price = sum(list(map(Album_Song_cost, Album_value)))
print('Price of all songs:', All_album_price, '$')

Average_album_price = All_album_price / Count_Of_Album_Songs
print('Average price of songs:', Average_album_price, '$')

Median_album_cost = list(map(Album_Song_cost, Album_value))
import statistics
print('Median of songs cost:', statistics.median(Median_album_cost), '$')

Expensive_album_song = max(Album_value, key=Album_Song_cost)
print('Expensive song:', Expensive_album_song['trackName'], Expensive_album_song['trackPrice'],'$')

Cheap_album_song = min(Album_value, key=Album_Song_cost)
if Cheap_album_song['trackPrice'] == -1 : 
    print('Cheap song:', Cheap_album_song['trackName'],'Free')
else:
    print('Cheap song:', Cheap_album_song['trackName'], Cheap_album_song['trackPrice'], '$')

Price of all songs: 11.61 $
Average price of songs: 1.29 $
Median of songs cost: 1.29 $
Expensive song: Piano Man 1.29 $
Cheap song: Piano Man 1.29 $
